In [8]:
#Importar bibliotecas
import pandas as pd
import numpy as np
import psycopg2 as pg2
import warnings
warnings.filterwarnings("ignore")

In [9]:
#Conectando com o PostgreSQL
conn = pg2.connect(database='DP6', user='postgres', password='200613')
cur=conn.cursor()
#Query no PostgreSQL
cur.execute('''
    SELECT q0.fullvisitorid, q0.receita,q0.qt_de_transações,
    q0.media_por_transação,q1.qt_de_sessões_conversão,q2.qt_de_sessões
    FROM (SELECT fullvisitorid, 
        SUM(revenue) AS Receita,
        COUNT(fullvisitorid) AS qt_de_transações,
        AVG(revenue) AS media_por_transação
        FROM hits
        GROUP BY (fullvisitorid)) 
        AS q0
    INNER JOIN (SELECT sessions.fullvisitorid, 
        COUNT(DISTINCT sessions.visitnumber) AS qt_de_sessões_conversão
        FROM sessions
        INNER JOIN hits
        ON hits.fullvisitorid = sessions.fullvisitorid
        WHERE hits.visitid = sessions.visitid 
        GROUP BY sessions.fullvisitorid) 
        AS q1
    ON q0.fullvisitorid = q1.fullvisitorid
    LEFT OUTER JOIN (SELECT fullvisitorid,
        COUNT(visitid) AS qt_de_sessões
        FROM sessions
        GROUP BY fullvisitorid) 
        AS q2
    ON q0.fullvisitorid = q2.fullvisitorid
    ORDER BY q0.receita DESC
    ''')
#Fechando conexão
data_base=cur.fetchall()
consulta=pd.DataFrame(data_base,columns=['fullvisitorid', 'Receita','Qt_de_transações',
'Media_por_transação','Qt_de_sessões_conversão','Qt_de_sessões'])
cur.close()
conn.commit()

In [10]:
#Exportar como excel
writer = pd.ExcelWriter('ConsultaSQL.xlsx', engine='xlsxwriter')
#Escrevendo guias
consulta.to_excel(writer,sheet_name='Consulta',index=False)
#Salvando
writer.save()